In [1]:
import os

from PIL import Image
from matplotlib.pyplot import imshow

import main
from Marble import Marble

MARBLE_BY_SYMBOL = dict(zip([Marble.symbol(e) for e in Marble], [e.name for e in Marble]))

status = main.init_image(Image.open(os.path.join("sample", "1.png")).convert('LA'))
print(status)


/Users/tsukatsuki/.pyenv/versions/anaconda3-5.0.1/envs/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a1c5b4e48>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './mnist_training'}


Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.


INFO:tensorflow:Restoring parameters from ./mnist_training/model.ckpt-1000


     e Q a v e a 
    - - - - a w - 
   - Q s f L s - - 
  - - w - a - w - - 
 - - e w s - e I a - 
f m Q - - G e - a - Q 
 f Q m w f - f f - v 
  S - w - w - m - a 
   e - f v a f T e 
    v - s - - w m 
     C - - - - e 


In [2]:
print("def neighbors")
x, y = 0, 0
result = []
for (dx, dy) in [(0, -1), (1, 0), (1, 1), (0, 1), (-1, 0), (-1, -1)]:
    n = (x + dx, y + dy)
    if n in status.state:
        result.append(status.state[n])
    else:
        result.append('-')
print(result)


def neighbors
['-', 'Q', '-', '-', '-', '-']


In [3]:
import numpy as np

print("def free")
neg = np.repeat(status.neighbors(x, y), 2)
xs = np.hstack(neg)
print(xs)
print(list(zip(*(xs[i:] for i in range(3)))))
print(('-', '-', '-') in list(zip(*(xs[i:] for i in range(3)))))


def free
['-' '-' 'Q' 'Q' '-' '-' '-' '-' '-' '-' '-' '-']
[('-', '-', 'Q'), ('-', 'Q', 'Q'), ('Q', 'Q', '-'), ('Q', '-', '-'), ('-', '-', '-'), ('-', '-', '-'), ('-', '-', '-'), ('-', '-', '-'), ('-', '-', '-'), ('-', '-', '-')]
True


In [4]:
print("def frees")
result = []
for (x, y) in status.state:
    if status.free(x, y):
        result.append((x, y))
print(result)


def frees
[(0, 0), (5, 0), (1, 2), (8, 4), (0, 5), (10, 5), (6, 9), (5, 10), (10, 10)]


In [5]:
def step(state):
    # for Quintessence use
    buckets = {}
    frees = sorted([(Marble[MARBLE_BY_SYMBOL[state.state[x]]], x) for x in state.frees()])
    for (k, v) in frees:
        buckets.setdefault(k, []).append(v)
    print("frees")
    for a in frees:
        (marbleA, posA) = a
        for b in frees:
            (marbleB, posB) = b
            if a == b:
                continue
            elif marbleA.value in range(Marble.Salt.value, Marble.Earth.value + 1):
                if marbleB == marbleA or marbleB == Marble.Salt:
                    yield {posA, posB}
            elif marbleA.value in range(Marble.Vitae.value, Marble.Mors.value + 1):
                if marbleB.value in range(Marble.Vitae.value, Marble.Mors.value + 1) and marbleA != marbleB:
                    yield {posA, posB}
            elif marbleA.value in range(Marble.Tin.value, Marble.Silver.value + 1):
                if marbleB == Marble.Quicksilver and marbleA.previous() not in state.keyState:
                    yield {posA, posB}
            elif marbleA == Marble.Lead and marbleB == Marble.Quicksilver:
                yield {posA, posB}
            elif marbleA == Marble.Gold:
                yield {posA}
            elif marbleA == Marble.Quintessence:
                continue
            elif marbleA == Marble.Quicksilver:
                continue
            else:
                print(marbleA, marbleB)

In [6]:
from copy import deepcopy

def solve(self):
    todo = [self]
    solutions = {str(self.state): []}
    print(solutions)
    print(str(self.state))
    while len(solutions) > 0:
        # eval
        cur_state = sorted(todo, key=lambda x: dict.__len__)[0]
        todo.remove(cur_state)
        for _step in cur_state.step():
            state = deepcopy(cur_state)
            for pos in _step:
                state.state.pop(pos) if pos in state.state else ()
        
        print(solutions)
        print(str(self.state))
        # solutions.pop(str(self.state))
        print(len(solutions))
        break
        # cur_state = 
        # continue


solve(status)


{"{(0, 0): 'e', (1, 0): 'Q', (2, 0): 'a', (3, 0): 'v', (4, 0): 'e', (5, 0): 'a', (4, 1): 'a', (5, 1): 'w', (1, 2): 'Q', (2, 2): 's', (3, 2): 'f', (4, 2): 'L', (5, 2): 's', (2, 3): 'w', (4, 3): 'a', (6, 3): 'w', (2, 4): 'e', (3, 4): 'w', (4, 4): 's', (6, 4): 'e', (7, 4): 'I', (8, 4): 'a', (0, 5): 'f', (1, 5): 'm', (2, 5): 'Q', (5, 5): 'G', (6, 5): 'e', (8, 5): 'a', (10, 5): 'Q', (1, 6): 'f', (2, 6): 'Q', (3, 6): 'm', (4, 6): 'w', (5, 6): 'f', (7, 6): 'f', (8, 6): 'f', (10, 6): 'v', (2, 7): 'S', (4, 7): 'w', (6, 7): 'w', (8, 7): 'm', (10, 7): 'a', (3, 8): 'e', (5, 8): 'f', (6, 8): 'v', (7, 8): 'a', (8, 8): 'f', (9, 8): 'T', (10, 8): 'e', (4, 9): 'v', (6, 9): 's', (9, 9): 'w', (10, 9): 'm', (5, 10): 'C', (10, 10): 'e'}": []}
{(0, 0): 'e', (1, 0): 'Q', (2, 0): 'a', (3, 0): 'v', (4, 0): 'e', (5, 0): 'a', (4, 1): 'a', (5, 1): 'w', (1, 2): 'Q', (2, 2): 's', (3, 2): 'f', (4, 2): 'L', (5, 2): 's', (2, 3): 'w', (4, 3): 'a', (6, 3): 'w', (2, 4): 'e', (3, 4): 'w', (4, 4): 's', (6, 4): 'e', (7, 4):